In [1]:
import pandas as pd
import numpy as np

In [67]:
dataset_path = r"D:\UOM\L4S1\FYP\Datasets\content_and_collaborative_filter_data.csv"

df = pd.read_csv(dataset_path, sep=';')
#df = df.drop(columns=['Traveler Type'])

In [68]:
df = df.drop('Traveler Type', axis=1)

In [69]:
df.head()

,Travel Group,Budget,Accommodation,Activity Interest,Physical Activity Level,Experience Level,Preferred Destination Category
0,Traveling with teenagers (12-18),Budget/Backpacking,Hostels & guesthouses,Spiritual & religious visits,Less active,Have traveled occasionally,Beaches & Coastal Areas
1,Traveling with extended family (multi-generational),Mid-range,3 - 4 star hotels,Hiking & trekking,Less active,First-time traveler,Nature & Wildlife
2,Traveling with extended family (multi-generational),Mid-range,3 - 4 star hotels,Surfing & water sports,Moderately active,First-time traveler,Adventure & Unique Experiences
3,Traveling with teenagers (12-18),Budget/Backpacking,Hostels & guesthouses,Hiking & trekking,Less active,First-time traveler,Adventure & Unique Experiences
4,Traveling with extended family (multi-generational),Mid-range,Budget hotels & Airbnb,Wildlife safaris,Moderately active,First-time traveler,Nature & Wildlife


In [70]:
df = df.sample(n=10000, random_state=42)

In [74]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder

# Separate features and target
X = df.drop(columns=['Preferred Destination Category'])  # Replace with actual target column
y = df['Preferred Destination Category']

le = LabelEncoder()
#X['Gender'] = le.fit_transform(X['Gender'])

oe1 = OrdinalEncoder(categories=[['Solo traveler', 'Traveling with a partner', 'Traveling with friends', 
                                 'Traveling with young kids (under 12)', 
                                 'Traveling with teenagers (12-18)', 
                                 'Traveling with extended family (multi-generational)']])
X['Travel Group'] = oe1.fit_transform(X[['Travel Group']])

# Automatically get unique categories for each column to encode
for col in ['Budget', 'Accommodation', 'Activity Interest', 'Physical Activity Level', 'Experience Level']:
    categories = [sorted(df[col].unique())]
    oe = OrdinalEncoder(categories=categories)
    X[col] = oe.fit_transform(X[[col]])


#X['Country'] = le.fit_transform(X['Country'])

y = le.fit_transform(y)

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder

# Separate features and target
X = df.drop(columns=['Preferred Destination Category'])  # Replace with actual target column
y = df['Preferred Destination Category']

le = LabelEncoder()
X['Gender'] = le.fit_transform(X['Gender'])

oe = OrdinalEncoder(categories=[['18-30 years', '31-50 years', '51+ years']])
X['Age Group'] = oe.fit_transform(X[['Age Group']])

X['Country'] = le.fit_transform(X['Country'])

y = le.fit_transform(y)

KeyError: 'Gender'

In [75]:
# Show mapping between original and encoded 'Travel Group' values
original = df['Budget']
encoded = X['Budget']

mapping_df = pd.DataFrame({'Original': original, 'Encoded': encoded})
mapping_df.drop_duplicates().sort_values('Encoded')

,Original,Encoded
65509,Budget/Backpacking,0.0
47044,Luxury,1.0
70975,Mid-range,2.0


In [76]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB


model_params = {
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'criterion': ['gini', 'entropy'],
            'max_depth': [3, 5, 7, None],
            'min_samples_split': [2, 5, 10]
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [10,50,100]
        }
    },
    'gaussian_nb': {
        'model': GaussianNB(),
        'params': {}
    }
}

In [77]:
from sklearn.model_selection import GridSearchCV

scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False, n_jobs=-1)
    clf.fit(X, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
df

,model,best_score,best_params
0,decision_tree,0.5750,"{'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 10}"
1,random_forest,0.5779,{'n_estimators': 100}
2,gaussian_nb,0.4510,{}


In [29]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

model_params = {
    'knn': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']
        }
    },
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params': {
            'C': [10, 20, 30],
            'kernel': ['rbf', 'sigmoid', 'poly']
        }
    }
}

# For supervised models (KNN, SVM)
from sklearn.model_selection import GridSearchCV

scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, n_jobs=-1)
    clf.fit(X, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

results_df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
print(results_df)

# For KMeans (unsupervised)
kmeans_scores = []
for k in [2, 3, 4, 5, 6]:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X)
    inertia = kmeans.inertia_
    kmeans_scores.append({'n_clusters': k, 'inertia': inertia})

kmeans_df = pd.DataFrame(kmeans_scores)
print(kmeans_df)

  model  best_score                                        best_params
0   knn       0.482  {'metric': 'euclidean', 'n_neighbors': 7, 'wei...
1   svm       0.519                         {'C': 20, 'kernel': 'rbf'}
   n_clusters      inertia
0           2  1847.128042
1           3  1052.752316
2           4   826.874758
3           5   702.100596
4           6   602.545581
